<img src="tamu.png" border="0" alt="TAMU Logo" width="200">
<h1>Quantum Instruction Set Architecture Development using pyQuil</h1>
<h3>Presented by: Venkatakrishnan Sutharsan</h3>

<h2>Importing Required Packages and utilities</h2>

In [1]:
from pyquil import Program, get_qc
from pyquil.gates import *
from pyquil.quilbase import Declare
import numpy as np
from pyquil.quil import DefGate
import numpy as np
from math import pi

from pyquil.api import WavefunctionSimulator
wf_sim = WavefunctionSimulator()

def view_graph(p):
    print(p)

<h2>The below is program in Quil to declare a memory space named "ro" and then apply Pauli-X gate to Qubit 0 of the circuit and read it to that specific memory space ro[0]</h2>

In [23]:
p = Program()
ro = p.declare('ro', 'BIT', 1)
p += X(0)
p += MEASURE(0, ro[0])

In [24]:
view_graph(p)

DECLARE ro BIT[1]
X 0
MEASURE 0 ro[0]



In [25]:
qc = get_qc('1q-qvm')  # You can make any 'nq-qvm' this way for any reasonable 'n'
executable = qc.compile(p)
result = qc.run(executable)
bitstrings = result.get_register_map().get('ro')
print(bitstrings)

[[1]]


<h2>Creating a sample circuit with Bell State to Demo Quantum Circuits</h2>

In [26]:
p = Program(
    Declare("ro", "BIT", 2),
    H(0),                         # Applying Hadamard Gate to Qubit 0 in the circuit
    CNOT(0, 1),                   # Applying CNOT (Controlled-NOT gate to bit 0 and 1 where bit 0 is the control bit) to the result of Hadamard Gate
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1)),
).wrap_in_numshots_loop(10) # We are running 10 iterations

In [27]:
print(p)

DECLARE ro BIT[2]
H 0
CNOT 0 1
MEASURE 0 ro[0]
MEASURE 1 ro[1]



The Quil language primitive shows that we do the following:
1. Create a Qubit named "ro" which is of type "BIT" and is 2-bits
2. Apply Hadamard Gate to Qubit 0 of the circuit (Superposition)
3. Apply Controlled-NOT gate to Qubit 0 and 1 with Qubit 0 as control signal

The above circuit is the basic circuit to demonstrate the Quantum Entanglement!

In [28]:
# run the program on a QVM
qc = get_qc('9q-square-qvm')
result = qc.run(qc.compile(p)).get_register_map().get("ro")
print(result)

[[1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]]


We can see that for the 10 iterations we are able to see that Qubit 1 is equal to Qubit 0, thus proving the Quantum Entanglement by the Bell Circuit

In [29]:
p = Program(
    Declare("ro", "BIT", 2),
    X(1),
    H(0),                         # Applying Hadamard Gate to Qubit 0 in the circuit
    CNOT(0, 1),                   # Applying CNOT (Controlled-NOT gate to bit 0 and 1 where bit 0 is the control bit) to the result of Hadamard Gate
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1)),
).wrap_in_numshots_loop(10) # We are running 10 iterations
print(p)
# run the program on a QVM
qc = get_qc('9q-square-qvm')
result = qc.run(qc.compile(p)).get_register_map().get("ro")
print(result)

DECLARE ro BIT[2]
X 1
H 0
CNOT 0 1
MEASURE 0 ro[0]
MEASURE 1 ro[1]

[[1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]]


<h2>Defining our own gate (Its a square-root gate)</h2>

In [30]:
# A multi-qubit defgate example
x_gate_matrix = np.array(([0.0, 1.0], [1.0, 0.0]))
sqrt_x = np.array([[ 0.5+0.5j,  0.5-0.5j],
                [ 0.5-0.5j,  0.5+0.5j]])
x_sqrt_x = np.kron(x_gate_matrix, sqrt_x)

In [31]:
x_sqrt_x_definition = DefGate("X-SQRT-X", x_sqrt_x)
X_SQRT_X = x_sqrt_x_definition.get_constructor()

# Then we can use the new gate
p = Program(x_sqrt_x_definition, X_SQRT_X(0, 1))

In [32]:
view_graph(p)

DEFGATE X-SQRT-X AS MATRIX:
	0, 0, 0.5+0.5i, 0.5-0.5i
	0, 0, 0.5-0.5i, 0.5+0.5i
	0.5+0.5i, 0.5-0.5i, 0, 0
	0.5-0.5i, 0.5+0.5i, 0, 0

X-SQRT-X 0 1



In [248]:
wavefunc = wf_sim.wavefunction(p)

In [249]:
print(wavefunc)

(0.5+0.5j)|01> + (0.5-0.5j)|11>


<h2>Testing our 2-bit adder</h2>

In [47]:
# A fully connected QVM
number_of_qubits = 3
qc = get_qc(f"{number_of_qubits}q-qvm")

In [51]:
q0 = 0 # Input 1 of the AND Gate
q1 = 1
q2 = 2 

p = Program().wrap_in_numshots_loop(10)
p += Declare("q","BIT", 6)

# Data formatting to show demo
p += X(0)
# p += X(1)
# p += X(2)

p += MEASURE(q0, ("q", 0))
p += MEASURE(q1, ("q", 1))
p += MEASURE(q2, ("q", 2))

p += CCNOT(q0, q1, q2)
p += CNOT(q0, q1)
# p += MEASURE(0,
p += MEASURE(q0, ("q", 3))
p += MEASURE(q1, ("q", 4))
p += MEASURE(q2, ("q", 5))
view_graph(p)

DECLARE q BIT[6]
X 0
MEASURE 0 q[0]
MEASURE 1 q[1]
MEASURE 2 q[2]
CCNOT 0 1 2
CNOT 0 1
MEASURE 0 q[3]
MEASURE 1 q[4]
MEASURE 2 q[5]



In [52]:
result = qc.run(qc.compile(p)).get_register_map().get("q")
wavefunc = wf_sim.wavefunction(p)
print(wavefunc)

(1+0j)|011>


In [53]:
print(result)

[[1 0 0 1 1 0]
 [1 0 0 1 1 0]
 [1 0 0 1 1 0]
 [1 0 0 1 1 0]
 [1 0 0 1 1 0]
 [1 0 0 1 1 0]
 [1 0 0 1 1 0]
 [1 0 0 1 1 0]
 [1 0 0 1 1 0]
 [1 0 0 1 1 0]]


<h1><b>Creating Custom Instruction</b></h1>

In [8]:
# A fully connected QVM
number_of_qubits = 6
qc = get_qc(f"{number_of_qubits}q-qvm")
# Initialize a program
p = Program().wrap_in_numshots_loop(10)

In [9]:
def custom_instruction(p, instr, *param):
    # print(param)
    if (instr == "superposition"):
        """ Instruction to do superposition on a qubit mentioned in the call.
        Usage: custom_instruction(p, "superposition", "h0", 0)
        """
        p += Declare(param[0],"BIT", 1)
        p += H(param[1])
        p += MEASURE(param[1], (param[0], 0))
    elif (instr == "rotate"):
        """ Instruction to do rotation on a qubit mentioned by a certain angle.
        Usage: custom_instruction(p, "rotate", "X", 
        """
        if (param[0] == "X" or param[0] == "x"):
            p += RX(param[1], param[2])
        elif (param[0] == "Y" or param[0] == "y"):
            p += RY(param[1], param[2])
        elif (param[0] == "Z" or param[0] == "z"):
            p += RZ(param[1], param[2])
    elif (instr == "NEG" or instr == "neg"):
        """ Instruction to do negation of a Qubit specified by the user/compiler
        Usage: custom_instruction(p, "neg", 1) 
        """
        p += Declare(param[0],"BIT", 1)
        p += X(param[1])
        p += MEASURE(param[1], (param[0], 0))
    elif (instr == "ADD" or instr == "add"):
        """ Instruction to addition of 2 bits.
        Usage : custom_instruction(p, "add", "add_qubit", 0, 1, 2)
            Param[0] - Name of memory you want to instantiate for this instruction result to be stored.
            Param[1] - The Qubit Number you want to use as input 1.
            Param[2] - The Qubit Number you want to use as input 2.
        """
        p += Declare(param[0],"BIT", 2)
        p += CCNOT(param[1], param[2], param[3])
        p += CNOT(param[1], param[2])
        p += MEASURE(param[2], (param[0], 0))  # Sum
        p += MEASURE(param[3], (param[0], 1))  # Carry
    elif (instr == "QFT" or instr == "qft"):
        """ Instruction to perform Quantum Fourier Transform on given number of bits.
        NOTE: Use only with 3-Qubits..!
        Usage: custom_instruction(p, "QFT", "qft_result", [1, 2, 3])
        """
        num_qubits = len(param[1])
        q0 = param[1][0]
        q1 = param[1][1]
        q2 = param[1][2]
        p += Declare(param[0],"BIT", num_qubits)
        # Apply Hadamard to all qubits
        p += H(param[1][0])
        p += H(param[1][1])
        p += H(param[1][2])
        
        # Apply controlled phase rotations
        for i in range(2, 0, -1):
            for j in range(i):
                control_qubit = param[1][j]
                target_qubit = param[1][j+1]
                angle = pi / (2**i)
                p += CPHASE(angle, control_qubit, target_qubit)

        p += MEASURE(param[1][0], (param[0], 0))
        p += MEASURE(param[1][1], (param[0], 1))
        p += MEASURE(param[1][2], (param[0], 2))
        # Apply SWAP gate for bit-reversal order (optional)
        # p += SWAP(q0, q2)
    elif (instr == "MOV" or instr == "mov"):
        """ Instruction to move value from one qubit to another qubit
        Usage: custom_instruction(p, "MOV", 1, 2)
        """
        # Can also be done using 3-CNOT gate
        p += Declare(param[0],"BIT", 2)
        p += SWAP(param[1], param[2])
        p += MEASURE(param[1], (param[0], 0))
        p += MEASURE(param[2], (param[0], 1))

In [10]:
#  Applying Negate to Qubit 0 and 1
# custom_instruction(p, "NEG", "neg_result1", 0)
# custom_instruction(p, "NEG", "neg_result2", 1)
# custom_instruction(p, "NEG", "neg_result2", 2)
# custom_instruction(p, "NEG", "neg_result", 4)
# Applying Add operation to Qubit 0, 1 with Qubit 2 as 0 to get carry directly.
# custom_instruction(p, "ADD", "add_result1", 0, 1, 2)

# Negating Qubit 3 to get 1 basis and fo QFT on Qubits 3,4,5
custom_instruction(p, "NEG", "neg_result", 3)
custom_instruction(p, "QFT", "qft_result", [3, 4, 5])

In [11]:
view_graph(p)

DECLARE neg_result BIT[1]
DECLARE qft_result BIT[3]
X 3
MEASURE 3 neg_result[0]
H 3
H 4
H 5
CPHASE(0.7853981633974483) 3 4
CPHASE(0.7853981633974483) 4 5
CPHASE(1.5707963267948966) 3 4
MEASURE 3 qft_result[0]
MEASURE 4 qft_result[1]
MEASURE 5 qft_result[2]



In [12]:
result = qc.run(qc.compile(p)).get_register_map()
wavefunc = wf_sim.wavefunction(p)
print(wavefunc.amplitudes)
print(len(wavefunc.amplitudes))

[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
8


In [13]:
print(result)

{'qft_result': array([[1, 0, 0],
       [1, 0, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 1, 1],
       [0, 0, 1],
       [1, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 0]]), 'neg_result': array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])}


In [17]:
from numpy.fft import ifft
print(ifft(wavefunc.amplitudes, norm="ortho"))

[-0.125     +0.j         -0.03628558-0.11961754j  0.1039337 -0.06944628j
  0.09662631+0.07929916j -0.04783543+0.11548494j -0.12439809-0.01225214j
 -0.02438629-0.12259816j  0.11024016-0.05892459j  0.08838835+0.08838835j
 -0.05892459+0.11024016j -0.12259816-0.02438629j -0.01225214-0.12439809j
  0.11548494-0.04783543j  0.07929916+0.09662631j -0.06944628+0.1039337j
 -0.11961754-0.03628558j  0.        -0.125j       0.11961754-0.03628558j
  0.06944628+0.1039337j  -0.07929916+0.09662631j -0.11548494-0.04783543j
  0.01225214-0.12439809j  0.12259816-0.02438629j  0.05892459+0.11024016j
 -0.08838835+0.08838835j -0.11024016-0.05892459j  0.02438629-0.12259816j
  0.12439809-0.01225214j  0.04783543+0.11548494j -0.09662631+0.07929916j
 -0.1039337 -0.06944628j  0.03628558-0.11961754j  0.125     +0.j
  0.03628558+0.11961754j -0.1039337 +0.06944628j -0.09662631-0.07929916j
  0.04783543-0.11548494j  0.12439809+0.01225214j  0.02438629+0.12259816j
 -0.11024016+0.05892459j -0.08838835-0.08838835j  0.05892459